# Testing Nsaba Functionality

##### Current Methods
- est_aba_ge()
- ge_ratio()
- est_ns_act()
- make_ge_ns_mat()

NsabaVisualizer methods also tested.

In [1]:
from nsaba.nsaba import Nsaba
from nsaba.nsaba.builder import NsabaBuilder
from nsaba.nsaba.visualizer import NsabaVisualizer
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import itertools
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
%load_ext line_profiler

In [3]:
# Simon Path IO

data_dir = '../../data_dir'
os.chdir(data_dir)
Nsaba.aba_load()
Nsaba.ns_load()

This may take a minute or two ...
SampleAnnot.csv loaded.
MicroarrayExpression.csv loaded.
Probes.csv loaded.
Nsaba.aba['mni_coords'] initialized.

This may take a minute or two ...
database.txt loaded.
features.txt loaded.
Nsaba.ns['mni_coords'] initialized.



In [ ]:
# Loading gene expression for all ABA registered Entrez IDs.
A = Nsaba()
# A.load_ge_pickle('Nsaba_ABA_ge.pkl')

In [ ]:
entrids = [1813,1816]
coords = [(np.random.uniform(-50,50), np.random.uniform(-50,50), np.random.uniform(-50,50)) for _ in range(50)]
A.ge_ratio(entrids, np.array(coords), rnn_args={'radius':10})

In [ ]:
entrids = [1813,1816]
A.estimate_aba_ge(entrids, radius=10)

In [ ]:
%time A.estimate_ns_act('attention')
A.estimate_ns_act('reward')

## Visualization Methods (testing)

In [ ]:
NV = NsabaVisualizer(A)

In [ ]:
NV.visualize_ge([1813])

In [ ]:
NV.visualize_ns('attention', alpha=.3)

In [ ]:
NV.lstsq_ns_ge('attention', [1813]);

In [ ]:
NV.lstsq_ge_ge(1813, 1816);

In [ ]:
NV.lstsq_ns_ns('attention', 'reward');

In [ ]:
A.matrix_builder(['reward'], [1813,1816])